# HeadLess란?
Headless라는 용어는 '창이 없는'과 같다고 이해하시면 됩니다. 여러분이 브라우저(크롬 등)을 이용해 인터넷을 브라우징 할 때 기본적으로 창이 뜨고 HTML파일을 불러오고, CSS파일을 불러와 어떤 내용을 화면에 그러야 할지 계산을 하는 작업을 브라우저가 자동으로 진행해줍니다.

하지만 이와같은 방식을 사용할 경우 사용하는 운영체제에 따라 크롬이 실행이 될 수도, 실행이 되지 않을 수도 있습니다. 예를들어 우분투 서버와 같은 OS에서는 '화면' 자체가 존재하지 않기 때문에 일반적인 방식으로는 크롬을 사용할 수 없습니다. 이를 해결해 주는 방식이 바로 Headless 모드입니다. 브라우저 창을 실제로 운영체제의 '창'으로 띄우지 않고 대신 화면을 그려주는 작업(렌더링)을 가상으로 진행해주는 방법으로 실제 브라우저와 동일하게 동작하지만 창은 뜨지 않는 방식으로 동작할 수 있습니다.

# 그러면 왜 크롬?
일전 가이드에서 PhantomJS(팬텀)라는 브라우저를 이용하는 방법에 대해 다룬적이 있습니다. 팬텀은 브라우저와 유사하게 동작하고 Javascript를 동작시켜주지만 성능상의 문제점과 크롬과 완전히 동일하게 동작하지는 않는다는 문제점이 있습니다. 우리가 크롤러를 만드는 상황이 대부분 크롬에서 진행하고, 크롬의 결과물 그대로 가져오기 위해서는 브라우저도 크롬을 사용하는 것이 좋습니다.

하지만 여전히 팬텀이 가지는 장점이 있습니다. WebDriver Binary만으로 추가적인 설치 없이 환경을 만들 수 있다는 장점이 있습니다.

윈도우 기준 크롬 59, 맥/리눅스 기준 크롬 60버전부터 크롬에 Headless Mode가 정식으로 추가되어서 만약 여러분의 브라우저가 최신이라면 크롬의 Headless모드를 쉽게 이용할 수 있습니다.

# 크롬 버전 확인하기
크롬 버전 확인은 크롬 브라우저에서 chrome://version/로 들어가 확인할 수 있습니다.



이와 같이 크롬 버전이 60버전 이상인 크롬에서는 'Headless'모드를 사용할 수 있습니다.

# 기존 코드 수정하기
크롬의 헤드리스 모드를 사용하는 방식은 기존 selenium을 이용한 코드와 거의 동일합니다만, 몇가지 옵션을 추가해줘야합니다.

기존에 webdriver를 사용해 크롬을 동작한 경우 아래와 같은 코드를 사용할 수 있었습니다.

In [4]:
from selenium import webdriver 

# 유의: chromedriver를 위에서 받아준 
# chromdriver(windows는 chromedriver.exe)의 절대경로로 바꿔주세요!

driver = webdriver.Chrome('/Users/82106/Downloads/chromedriver_win32/chromedriver')

driver.get('http://naver.com')
driver.implicitly_wait(3)
driver.get_screenshot_as_file('naver_main.png')

driver.quit()

위 코드를 동작시키면 크롬이 켜지고 파이썬 파일 옆에 naver_main.png라는 스크린샷 하나가 생기게 됩니다. 이 코드는 지금까지 우리가 만들었던 코드와 큰 차이가 없는걸 확인해 보세요.

하지만 이 코드를 몇가지 옵션만 추가해주면 바로 Headless모드로 동작하게 만들어줄 수 있습니다.

In [9]:
from selenium import webdriver 

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("--disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

driver = webdriver.Chrome('/Users/82106/Downloads/chromedriver_win32/chromedriver', chrome_options=options)

driver.get('http://naver.com')
driver.implicitly_wait(3)
driver.get_screenshot_as_file('naver_main_headless.png')

driver.quit()

<ipython-input-9-464be76d5828>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/Users/82106/Downloads/chromedriver_win32/chromedriver', chrome_options=options)


위 코드를 보시면 ChromeOptions()를 만들어 add_argument를 통해 Headless모드인 것과, 크롬 창의 크기, 그리고 gpu(그래픽카드 가속)를 사용하지 않는 옵션을 넣어준 것을 볼 수 있습니다.

제일 중요한 부분은 바로 options.add_argument('headless')라는 부분입니다. 크롬이 Headless모드로 동작하도록 만들어주는 키워드에요. 그리고 크롬 창의 크기를 직접 지정해 준 이유는, 여러분이 일반적으로 노트북이나 데스크탑에서 사용하는 모니터의 해상도가 1920x1080이기 때문입니다. 즉, 여러분이 일상적으로 보는 것 그대로 크롬이 동작할거라는 기대를 해볼수 있습니다!

마지막으로는 disable-gpu인데요, 만약 위 코드를 실행했을때 GPU에러~가 난다면 --disable-gpu로 앞에 dash(-)를 두개 더 붙여보세요. 이 버그는 크롬 자체에 있는 문제점입니다. 브라우저들은 CPU의 부담을 줄이고 좀더 빠른 화면 렌더링을 위해 GPU를 통해 그래픽 가속을 사용하는데, 이 부분이 크롬에서 버그를 일으키는 현상을 보이고 있습니다. (윈도우 크롬 61버전까지는 아직 업데이트 되지 않았습니다. 맥 61버전에는 해결된 이슈입니다.)

그리고 driver 변수를 만들 때 단순하게 chromedriver의 위치만 적어주는 것이 아니라 chrome_options라는 이름의 인자를 함께 넘겨줘야 합니다.

이 chrome_options는 Chrome을 이용할때만 사용하는 인자인데요, 이 인자값을 통해 headless등의 추가적인 인자를 넘겨준답니다.

자, 이제 그러면 한번 실행해 보세요. 크롬 창이 뜨지 않았는데도 naver_main_headless.png파일이 생겼다면 여러분 컴퓨터에서 크롬이 Headless모드로 성공적으로 실행된 것이랍니다!

# Headless브라우저임을 숨기기
Headless모드는 CLI기반의 서버 OS에서도 Selenium을 통한 크롤링/테스트를 가능하게 만드는 멋진 모드지만, 어떤 서버들에서는 이런 Headless모드를 감지하는 여러가지 방법을 쓸 수 있습니다.

아래 글에서는 Headless모드를 탐지하는 방법과 탐지를 '막는'방법을 다룹니다.(창과 방패, 또 새로운 창!)

아래 코드의 TEST_URL은 https://intoli.com/blog/making-chrome-headless-undetectable/chrome-headless-test.html 인데요, 이곳에서 Headless모드가 감춰졌는지 아닌지 확인해 볼 수 있습니다.